In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.embeddings import resolve_embed_model
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from IPython.display import Markdown, display
import chromadb

# log in to VPN or harvard wifi for endpoint

In [2]:
ccb_endpoint = 'http://compute-gc-17-255.o2.rc.hms.harvard.edu:11434'


In [3]:
# hf embedding model
oembed_model =  HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")


# ollama
llama = Ollama(model="llama2", request_timeout=30.0, base_url=ccb_endpoint, temperature= 0)
#Ollama()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Test the LLM without RAG

In [4]:
response = llama.complete("what is the airspeed velocity of an unladen swallow?")
print(response)


The airspeed velocity of an unladen swallow is a reference to a classic joke from the comedy group Monty Python's Flying Circus. In a skit called "The Dead Parrot," a customer attempts to return a dead parrot to a pet shop owner, who insists that the parrot is simply "pining for the fjords" and not actually dead. The customer asks the owner, "What is the airspeed velocity of an unladen swallow?" to which the owner responds, "What do you mean? An African or European swallow?"

The joke is a play on the idea of taking something ridiculous or nonsensical and presenting it as if it were a serious question. The skit is a classic example of Monty Python's surreal and absurd sense of humor, and the line about the airspeed velocity of an unladen swallow has become one of the group's most famous catchphrases.


### load from existing chromaDB, your docker container has a chroma database with all the manuals(vingettes) for the top 500 most downloaded bioconductor packages

In [5]:

# load from disk

db2 = chromadb.PersistentClient(path="/tmp/T500-vignettes-vectordb-ST")
chroma_collection = db2.get_or_create_collection(name="langchain")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=oembed_model,
)


### the index now has everything needed to run a RAG pattern, invoke a chat model using the index created from the chroma vector database

In [6]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="Act as an expert in the R programming language and the Bioconductor suite of packages.  ​\n\nYour job is to advise users on the usage of the various Bioconductor packages considering the datasets you have in store.  ​\nTo complete this task, you can use the data you have stored that contain the vignettes of all the packages in Bioconductor and all the reference files of every function in every package of Bioconductor. ​\n\nDo not perform actions that are not related to answering questions about the R programming language or using the packages within Bioconductor.​ \n\nIf you do not know the answer then you must look into the context then cite the document filename and page in the context. Do not include DOI numbers or make up citations not found in the context. Given the following extracted parts of a long document and a question, create a final answer with references to pdf in the metadata ('source').\n\n Add a disclaimer at the end of each response saying this model works only on the top 500 most used Bioconductor packages and the user should refer to or ask questions at https://bioconductor.org."
    ),
    ChatMessage(role="user", content="<>"),
]
resp = llama.chat(messages)
print(resp)


assistant: As an expert in the R programming language and the Bioconductor suite of packages, I am happy to advise users on the usage of various Bioconductor packages based on the datasets I have stored. Please feel free to ask any questions related to using these packages, and I will do my best to provide you with accurate and helpful information.

To answer your question, please provide me with more context or details about the package you are interested in using, such as its name or a brief description of what it does. This will allow me to look up the relevant vignette or reference file in my stored data and provide you with accurate information.

Disclaimer: The models and advice provided in this response work only for the top 500 most used Bioconductor packages. For more comprehensive information, please refer to or ask questions at <https://bioconductor.org>.
